In [ ]:
from sqlalchemy import create_engine, Table, MetaData, select
import sqlite3

# Step 1: Connect to the SQLite database using sqlite3
conn = sqlite3.connect('books.db')
cursor = conn.cursor()

# Check if the books table exists
cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='table' AND name='books'")
table_exists = cursor.fetchone()

if not table_exists:
    print("The 'books' table does not exist. Creating it now...")
    cursor.execute('''
    CREATE TABLE books (
        title TEXT,
        author TEXT,
        year INTEGER
    )
    ''')
    print("'books' table created successfully.")

# Check if the books table has data
cursor.execute("SELECT * FROM books")
rows = cursor.fetchall()

if not rows:  # Insert sample data if the table is empty
    print("No data found in 'books' table. Inserting sample data...")
    sample_data = [
        ('1984', 'George Orwell', 1949),
        ('Pride and Prejudice', 'Jane Austen', 1813),
        ('To Kill a Mockingbird', 'Harper Lee', 1960)
    ]
    cursor.executemany(
        "INSERT INTO books (title, author, year) VALUES (?, ?, ?)", sample_data)
    conn.commit()
    print("Sample data inserted successfully.")

# Display data in the books table
print("\nData in 'books' table:")
cursor.execute("SELECT * FROM books")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the sqlite3 connection
conn.close()

# Step 2: Connect to the SQLite database using SQLAlchemy
engine = create_engine('sqlite:///books.db')
connection = engine.connect()

# Reflect the books table using SQLAlchemy
metadata = MetaData()
books = Table('books', metadata, autoload_with=engine)

# Query the title column in alphabetical order
query = select(books.c.title).order_by(books.c.title)
result = connection.execute(query)

# Print the titles in alphabetical order
print("\nTitles in alphabetical order:")
for row in result:
    print(row.title)

# Close the SQLAlchemy connection
connection.close()
